In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2024-01-17 22:54:21.547069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'ASK_AMT' and 'SPECIAL_CONSIDERSATIONS'.
application_df = application_df.drop(columns=["EIN", "NAME", "ASK_AMT", "SPECIAL_CONSIDERATIONS"])

# Display dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts.iloc[8:].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_types = application_df['CLASSIFICATION'].value_counts()
classification_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_types.iloc[5:].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,1,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [8]:
# Split our preprocessed data into our features and target arrays
y = dummies_df["IS_SUCCESSFUL"].values
X = dummies_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt 1 (73.54%)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 328       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 379 (1.48 KB)
Trainable params: 379 (1.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6457 - accuracy: 0.6597
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5752 - accuracy: 0.7270
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5663 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5626 - accuracy: 0.7269
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5602 - accuracy: 0.7269
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5588 - accuracy: 0.7259
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5574 - accuracy: 0.7264
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5556 - accuracy: 0.7278
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5544 - accuracy: 0.7287
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accura

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5490 - accuracy: 0.7354 - 487ms/epoch - 2ms/step
Loss: 0.5489735007286072, Accuracy: 0.7353935837745667


# Second Attempt (73.26%)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 328       
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 409 (1.60 KB)
Trainable params: 409 (1.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6135 - accuracy: 0.6829
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5712 - accuracy: 0.7170
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5648 - accuracy: 0.7199
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5616 - accuracy: 0.7216
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5599 - accuracy: 0.7260
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5585 - accuracy: 0.7257
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5573 - accuracy: 0.7268
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7278
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7290
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5507 - accuracy: 0.7326 - 316ms/epoch - 1ms/step
Loss: 0.5507398843765259, Accuracy: 0.7325947284698486


# Attempt 3 (73.31%)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 7
hidden_nodes_layer3 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 9)                 369       
                                                                 
 dense_7 (Dense)             (None, 7)                 70        
                                                                 
 dense_8 (Dense)             (None, 3)                 24        
                                                                 
 dense_9 (Dense)             (None, 1)                 4         
                                                                 
Total params: 467 (1.82 KB)
Trainable params: 467 (1.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6350 - accuracy: 0.6598
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5891 - accuracy: 0.7238
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5757 - accuracy: 0.7261
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5681 - accuracy: 0.7274
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5635 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5611 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5592 - accuracy: 0.7290
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5579 - accuracy: 0.7278
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5571 - accuracy: 0.7293
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5524 - accuracy: 0.7331 - 441ms/epoch - 2ms/step
Loss: 0.5523717999458313, Accuracy: 0.7330612540245056
